# 1. Imports & Global config

In [ ]:
import os, math
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, make_scorer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import joblib

# Reusable settings
RANDOM_STATE = 42
N_SPLITS = 5
TARGET = "total_emission"
BASE_DIR = Path(r"C:\Users\Dewald\Documents\Regression Project")  # <-- your path
xlsx_path = BASE_DIR / "co2_emissions_from_agri_cleaned.xlsx"
csv_path  = BASE_DIR / "co2_emissions_from_agri_cleaned.csv"

def rmsle_safe(y_true, y_pred, eps=1e-9):
    y_true = np.clip(np.asarray(y_true), a_min=eps, a_max=None)
    y_pred = np.clip(np.asarray(y_pred), a_min=eps, a_max=None)
    try:
        return math.sqrt(mean_squared_log_error(y_true, y_pred))
    except ValueError:
        return np.nan

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def cv_scorers():
    # negative RMSE so GridSearchCV can "maximize"
    return {
        "rmse": make_scorer(lambda yt, yp: rmse(yt, yp), greater_is_better=False),
        "r2": "r2"
    }

def print_test_report(name, y_true, y_pred):
    print(f"\n{name} — Test set metrics")
    print(f"  RMSE : {rmse(y_true, y_pred):.6f}")
    print(f"  R²   : {r2_score(y_true, y_pred):.6f}")
    print(f"  RMSLE: {rmsle_safe(y_true, y_pred):.6f}")

def extract_importances_or_coefs(pipe, feature_names):
    """Return DataFrame with ['feature','importance'] if available; else None."""
    reg = pipe[-1] if not hasattr(pipe, "named_steps") else pipe.named_steps.get("reg", None)
    if reg is None:
        return None
    # LinearRegression => absolute coefficients (on scaled features)
    if isinstance(reg, LinearRegression):
        coef = getattr(reg, "coef_", None)
        if coef is None:
            return None
        coefs = np.abs(np.ravel(coef))
        return pd.DataFrame({"feature": feature_names, "importance": coefs}).sort_values("importance", ascending=False)
    # Tree-based => feature_importances_
    fi = getattr(reg, "feature_importances_", None)
    if fi is None:
        return None
    return pd.DataFrame({"feature": feature_names, "importance": fi}).sort_values("importance", ascending=False)


# 2. Load & basic cleanup

In [4]:
if xlsx_path.exists():
    print(f"Loading Excel: {xlsx_path}")
    df = pd.read_excel(xlsx_path, sheet_name=0)
elif csv_path.exists():
    print(f"Loading CSV:   {csv_path}")
    df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError(
        f"Could not find data file in:\n - {xlsx_path}\n - {csv_path}\n"
        "Place your cleaned dataset there with one of these names."
    )

# Drop unnamed helper columns
for col in list(df.columns):
    if str(col).lower().startswith("unnamed"):
        df.drop(columns=[col], inplace=True)

assert TARGET in df.columns, f"Expected target column '{TARGET}' in columns."

# Keep numeric features (excluding target)
numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c]) and c != TARGET]
if not numeric_cols:
    raise ValueError("No numeric feature columns (besides target) were found.")

X = df[numeric_cols].copy()
y = df[TARGET].copy()

# Clean NaNs
mask = y.notna()
X, y = X.loc[mask], y.loc[mask]
X = X.dropna(axis=0)
y = y.loc[X.index]

print(f"Data ready. Rows: {len(df):,}; Numeric features used: {len(numeric_cols)}")


Loading Excel: C:\Users\Dewald\Documents\Regression Project\co2_emissions_from_agri_cleaned.xlsx
Data ready. Rows: 6,965; Numeric features used: 28


# 3. Train / Test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE
)
feature_names = list(X_train.columns)

print(f"Train size: {X_train.shape[0]:,} | Test size: {X_test.shape[0]:,}")


Train size: 5,572 | Test size: 1,393


# 4. Linear Regression + CV

In [8]:
linr_pipe = Pipeline([
    ("prep", ColumnTransformer([("num", StandardScaler(), feature_names)], remainder="drop")),
    ("reg", LinearRegression())
])

cv = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
scores_linr = cross_validate(
    linr_pipe, X_train, y_train,
    scoring=cv_scorers(), cv=cv, return_train_score=False
)
print("LinearRegression — CV results")
print(f"  RMSE (mean±sd): {(-scores_linr['test_rmse']).mean():.6f} ± {(-scores_linr['test_rmse']).std():.6f}")
print(f"  R²   (mean±sd): { (scores_linr['test_r2']).mean():.6f} ± { (scores_linr['test_r2']).std():.6f}")

linr_pipe.fit(X_train, y_train)
linr_pred = linr_pipe.predict(X_test)
print_test_report("LinearRegression", y_test, linr_pred)

# Save artifacts for this model
OUT_DIR = BASE_DIR
linr_pred_path = OUT_DIR / "linear_regression_predictions_test.csv"
pd.DataFrame({"y_true": y_test.values, "y_pred": linr_pred, "residual": y_test.values - linr_pred},
             index=y_test.index).to_csv(linr_pred_path)

linr_model_path = OUT_DIR / "model_linear_regression.joblib"
joblib.dump(linr_pipe, linr_model_path, compress=3)

print("Saved:")
print(" ", linr_pred_path)
print(" ", linr_model_path)


LinearRegression — CV results
  RMSE (mean±sd): 1800885.499945 ± 281991.562235
  R²   (mean±sd): 0.997797 ± 0.000562

LinearRegression — Test set metrics
  RMSE : 1978461.775516
  R²   : 0.998382
  RMSLE: 7.424983
Saved:
  C:\Users\Dewald\Documents\Regression Project\linear_regression_predictions_test.csv
  C:\Users\Dewald\Documents\Regression Project\model_linear_regression.joblib


# 4a. Random Forest + GridSearchCV

In [10]:
# === D2) Random Forest + GridSearchCV ===
rf_pipe = Pipeline([
    ("prep", "passthrough"),  # trees don't need scaling
    ("reg", RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

rf_param_grid = {
    "reg__n_estimators": [200, 400],
    "reg__max_depth": [None, 10, 20],
    "reg__min_samples_leaf": [1, 2, 4]
}

cv = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
rf_grid = GridSearchCV(
    rf_pipe, rf_param_grid, cv=cv,
    scoring=cv_scorers(), refit="rmse",  # refit on negative-RMSE scorer
    n_jobs=-1, verbose=1
)
rf_grid.fit(X_train, y_train)
print("\nRandomForest — best CV params:", rf_grid.best_params_)

rf_best = rf_grid.best_estimator_
rf_pred = rf_best.predict(X_test)
print_test_report("RandomForest (best grid)", y_test, rf_pred)

# Save artifacts for this model
OUT_DIR = BASE_DIR
rf_pred_path = OUT_DIR / "random_forest_predictions_test.csv"
pd.DataFrame({"y_true": y_test.values, "y_pred": rf_pred, "residual": y_test.values - rf_pred},
             index=y_test.index).to_csv(rf_pred_path)

rf_model_path = OUT_DIR / "model_random_forest.joblib"
joblib.dump(rf_best, rf_model_path, compress=3)

print("Saved:")
print(" ", rf_pred_path)
print(" ", rf_model_path)


Fitting 5 folds for each of 18 candidates, totalling 90 fits

RandomForest — best CV params: {'reg__max_depth': None, 'reg__min_samples_leaf': 1, 'reg__n_estimators': 400}

RandomForest (best grid) — Test set metrics
  RMSE : 235861.154027
  R²   : 0.999977
  RMSLE: 0.456070
Saved:
  C:\Users\Dewald\Documents\Regression Project\random_forest_predictions_test.csv
  C:\Users\Dewald\Documents\Regression Project\model_random_forest.joblib


# 4b. Gradient Boosting + GridSearchCV

In [12]:
gb_pipe = Pipeline([
    ("prep", "passthrough"),
    ("reg", GradientBoostingRegressor(random_state=RANDOM_STATE))
])

gb_param_grid = {
    "reg__n_estimators": [200, 400],
    "reg__learning_rate": [0.05, 0.1],
    "reg__max_depth": [2, 3],
    "reg__min_samples_leaf": [1, 3]
}

cv = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
gb_grid = GridSearchCV(
    gb_pipe, gb_param_grid, cv=cv,
    scoring=cv_scorers(), refit="rmse",
    n_jobs=-1, verbose=1
)
gb_grid.fit(X_train, y_train)
print("\nGradientBoosting — best CV params:", gb_grid.best_params_)

gb_best = gb_grid.best_estimator_
gb_pred = gb_best.predict(X_test)
print_test_report("GradientBoosting (best grid)", y_test, gb_pred)

# Save artifacts for this model
OUT_DIR = BASE_DIR
gb_pred_path = OUT_DIR / "gradient_boosting_predictions_test.csv"
pd.DataFrame({"y_true": y_test.values, "y_pred": gb_pred, "residual": y_test.values - gb_pred},
             index=y_test.index).to_csv(gb_pred_path)

gb_model_path = OUT_DIR / "model_gradient_boosting.joblib"
joblib.dump(gb_best, gb_model_path, compress=3)

print("Saved:")
print(" ", gb_pred_path)
print(" ", gb_model_path)


Fitting 5 folds for each of 16 candidates, totalling 80 fits

GradientBoosting — best CV params: {'reg__learning_rate': 0.1, 'reg__max_depth': 2, 'reg__min_samples_leaf': 1, 'reg__n_estimators': 400}

GradientBoosting (best grid) — Test set metrics
  RMSE : 245403.746863
  R²   : 0.999975
  RMSLE: 1.120372
Saved:
  C:\Users\Dewald\Documents\Regression Project\gradient_boosting_predictions_test.csv
  C:\Users\Dewald\Documents\Regression Project\model_gradient_boosting.joblib


# 5. Compare models (test set)

In [14]:
# This expects the variables linr_pred / rf_pred / gb_pred to exist depending on what you've run.
results = []

if 'linr_pred' in globals():
    results.append(("LinearRegression", rmse(y_test, linr_pred), r2_score(y_test, linr_pred), rmsle_safe(y_test, linr_pred)))
if 'rf_pred' in globals():
    results.append(("RandomForest", rmse(y_test, rf_pred), r2_score(y_test, rf_pred), rmsle_safe(y_test, rf_pred)))
if 'gb_pred' in globals():
    results.append(("GradientBoosting", rmse(y_test, gb_pred), r2_score(y_test, gb_pred), rmsle_safe(y_test, gb_pred)))

if not results:
    raise RuntimeError("No models have been run yet. Execute one or more of D1-D3 first.")

results_sorted = sorted(results, key=lambda r: r[1])  # by RMSE asc
print("\n=== Model leaderboard (by Test RMSE) ===")
for name, rmse_v, r2_v, rmsle_v in results_sorted:
    print(f"{name:<18}  RMSE={rmse_v:.6f} | R²={r2_v:.6f} | RMSLE={rmsle_v:.6f}")



=== Model leaderboard (by Test RMSE) ===
RandomForest        RMSE=235861.154027 | R²=0.999977 | RMSLE=0.456070
GradientBoosting    RMSE=245403.746863 | R²=0.999975 | RMSLE=1.120372
LinearRegression    RMSE=1978461.775516 | R²=0.998382 | RMSLE=7.424983


# 6. Save the single best model

In [16]:
# === F) Save the single best model (of the ones you've run) ===
# Recompute 'results_with_models' using whichever models you've already run:
results_with_models = []

if 'linr_pipe' in globals() and 'linr_pred' in globals():
    results_with_models.append(("LinearRegression", rmse(y_test, linr_pred), r2_score(y_test, linr_pred), rmsle_safe(y_test, linr_pred), linr_pipe))
if 'rf_best' in globals() and 'rf_pred' in globals():
    results_with_models.append(("RandomForest", rmse(y_test, rf_pred), r2_score(y_test, rf_pred), rmsle_safe(y_test, rf_pred), rf_best))
if 'gb_best' in globals() and 'gb_pred' in globals():
    results_with_models.append(("GradientBoosting", rmse(y_test, gb_pred), r2_score(y_test, gb_pred), rmsle_safe(y_test, gb_pred), gb_best))

if not results_with_models:
    raise RuntimeError("No trained models found. Run D1–D3 first.")

results_with_models.sort(key=lambda r: r[1])  # by RMSE
best_name, best_rmse, best_r2, best_rmsle, best_model = results_with_models[0]
print(f"Best model so far: {best_name} (RMSE={best_rmse:.6f}, R²={best_r2:.6f}, RMSLE={best_rmsle:.6f})")

OUT_DIR = BASE_DIR
best_model_path = OUT_DIR / f"best_model_{best_name.lower()}.joblib"
joblib.dump(best_model, best_model_path, compress=3)
print("Saved best model to:", best_model_path)

# Also save a unified metrics CSV of only the models you've run in this session
metrics_df = pd.DataFrame([{
    "model": name, "rmse": rmse_v, "r2": r2_v, "rmsle": rmsle_v
} for name, rmse_v, r2_v, rmsle_v, _ in results_with_models])
metrics_path = OUT_DIR / "model_metrics_cv_test_session.csv"
metrics_df.to_csv(metrics_path, index=False)
print("Saved metrics to:", metrics_path)


Best model so far: RandomForest (RMSE=235861.154027, R²=0.999977, RMSLE=0.456070)
Saved best model to: C:\Users\Dewald\Documents\Regression Project\best_model_randomforest.joblib
Saved metrics to: C:\Users\Dewald\Documents\Regression Project\model_metrics_cv_test_session.csv


# 7. Feature importances / coefficients (Top 20)

In [18]:
# === G) Feature importances / coefficients (Top 20) ===
fi_df = extract_importances_or_coefs(best_model, feature_names)
if fi_df is not None and not fi_df.empty:
    top20 = fi_df.head(20).copy()
    top20_path = BASE_DIR / f"{best_name.lower()}_feature_importances_top20.csv"
    top20.to_csv(top20_path, index=False)
    print("Saved top 20 importances/coefficients to:", top20_path)
else:
    print("No feature importances/coefficients available for this model.")


Saved top 20 importances/coefficients to: C:\Users\Dewald\Documents\Regression Project\randomforest_feature_importances_top20.csv
